# Torrent Downloader

Paste your magnet link and run the cell.

In [ ]:
import sys, subprocess, time, os
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'libtorrent==2.0.11'])
import libtorrent as lt

MAGNET_LINK = ''
SAVE_PATH = '/content/torrents'

if not MAGNET_LINK:
    print('⚠️ Set MAGNET_LINK variable above')
else:
    os.makedirs(SAVE_PATH, exist_ok=True)
    trackers = ['udp://tracker.opentrackr.org:1337/announce', 'udp://open.stealth.si:80/announce', 'udp://tracker.torrent.eu.org:451/announce', 'udp://exodus.desync.com:6969/announce', 'udp://tracker.moeking.me:6969/announce', 'udp://explodie.org:6969/announce', 'udp://tracker1.bt.moack.co.kr:80/announce', 'udp://tracker.theoks.net:6969/announce']
    magnet_with_trackers = MAGNET_LINK + ''.join(f'&tr={tr}' for tr in trackers)
    session = lt.session({'enable_dht': True, 'enable_lsd': True, 'enable_upnp': True, 'enable_natpmp': True, 'connections_limit': 200, 'dht_announce_interval': 30})
    session.add_dht_router('router.bittorrent.com', 6881)
    session.add_dht_router('router.utorrent.com', 6881)
    params = lt.parse_magnet_uri(magnet_with_trackers)
    params.save_path = SAVE_PATH
    handle = session.add_torrent(params)
    print('📡 Fetching metadata...')
    timeout = 300
    start = time.time()
    while not handle.status().has_metadata:
        if time.time() - start > timeout:
            print('\n⚠️ Metadata fetch timeout')
            session.remove_torrent(handle)
            raise TimeoutError('Could not fetch metadata')
        time.sleep(1)
    print(f'✅ {handle.status().name}')
    print(f'📦 Size: {handle.status().total_wanted/(1024**3):.2f} GB')
    print('⬇️ Downloading...')
    while not handle.status().is_seeding:
        s = handle.status()
        print(f'\r{s.progress*100:.1f}% | ↓{s.download_rate/1024:.0f} KB/s | Peers: {s.num_peers}', end='')
        time.sleep(2)
    print('\n🎉 Download complete!')
    print(f'📁 Files saved to: {SAVE_PATH}')
    session.remove_torrent(handle)